In [1]:
# importacion de librerias necesarias
import numpy as np
import pandas as pd

* lectura de archivos :

In [2]:
bloomin = pd.read_parquet(r'CLEAN_reviews_bloomin.parquet')
brinker = pd.read_parquet(r'CLEAN_reviews_brinker.parquet')
texas = pd.read_parquet(r'CLEAN_reviews_texasroadhouse.parquet')
darden = pd.read_parquet(r'CLEAN_reviews_darden.parquet')

In [3]:
darden.columns

Index(['name', 'review_id', 'user_id', 'business_id', 'stars', 'text', 'date',
       'day', 'text_reply', 'city', 'state', 'postal_code', 'coordinates'],
      dtype='object')

In [4]:
texas.columns

Index(['name', 'review_id', 'user_id', 'business_id', 'stars', 'text', 'date',
       'day', 'city', 'state', 'postal_code', 'coordinates'],
      dtype='object')

In [5]:
brinker.columns

Index(['name', 'review_id', 'user_id', 'business_id', 'stars', 'text', 'date',
       'day', 'city', 'state', 'postal_code', 'coordinates'],
      dtype='object')

In [6]:
bloomin.columns

Index(['name', 'review_id', 'user_id', 'business_id', 'stars', 'text', 'date',
       'day', 'city', 'state', 'postal_code', 'coordinates'],
      dtype='object')

* preparación de los datos para calculo de KPI

In [7]:
# convierto a datetime la columna de fecha
darden['date'] = pd.to_datetime(darden['date'])

In [8]:
# crear columna adicional para dividir los trimestres, el año entonces se dividira en quarters 
darden['quarter'] = darden['date'].dt.to_period('Q')

# agrupamos por 'quarter' y calculamos la puntuación promedio
promedio_estrellas_per_quarter = darden.groupby('quarter')['stars'].mean().reset_index()
promedio_estrellas_per_quarter.columns = ['quarter', 'promedio_stars']

In [9]:
darden.head(2)

,name,review_id,user_id,business_id,stars,text,date,day,text_reply,city,state,postal_code,coordinates,quarter
0,Cheddar's Scratch Kitchen,_nM2IN_PleArpBRKzbq2Jg,G5nBrEuXyVMYPwD9cVtN4g,3kCdCzbfMbpTStmrUbpfUQ,1.0,The service was terrible. My wife's meal came ...,2019-09-18,Wednesday,,Tampa,PA,33618,"28.048990114,-82.505071509",2019Q3
1,Yard House,3D_YF1igbVDUr23jmjrSyQ,9mk2G2NK6UbShCpWYP_79A,bSX7czEHT7plB0JG_2aPHg,2.0,"Kitchen was very slow, 2 out of 8 meals serve...",2020-02-17,Monday,,Willow Grove,PA,19090,"40.1402883373,-75.1249255505",2020Q1


In [10]:
darden['is_negative'] = darden['stars'] <= 3

# Contamos la cantidad de reseñas negativas y positivas
negative_reviews_count = darden['is_negative'].sum()
positive_reviews_count = (~darden['is_negative']).sum()

print(f"Cantidad de reseñas negativas: {negative_reviews_count}")
print(f"Cantidad de reseñas positivas: {positive_reviews_count}")

Cantidad de reseñas negativas: 2757
Cantidad de reseñas positivas: 4396


In [11]:
state_reviews_count = darden.groupby(['state', 'is_negative']).size().unstack(fill_value=0).reset_index()

# Renombramos las columnas para mayor claridad
state_reviews_count.columns = ['state', 'reviews_positivas', 'reviews_negativas']

state_reviews_count

,state,reviews_positivas,reviews_negativas
0,FL,1515,995
1,NJ,250,247
2,PA,2631,1515


In [12]:
bloomin['is_negative'] = bloomin['stars'] <= 3

# Contamos la cantidad de reseñas negativas y positivas
negative_reviews_count = bloomin['is_negative'].sum()
positive_reviews_count = (~bloomin['is_negative']).sum()

print(f"Cantidad de reseñas negativas bloomin: {negative_reviews_count}")
print(f"Cantidad de reseñas positivas bloomin: {positive_reviews_count}")
print('------')


texas['is_negative'] = texas['stars'] <= 3

# Contamos la cantidad de reseñas negativas y positivas
negative_reviews_count = texas['is_negative'].sum()
positive_reviews_count = (~texas['is_negative']).sum()

print(f"Cantidad de reseñas negativas texas: {negative_reviews_count}")
print(f"Cantidad de reseñas positivas texas: {positive_reviews_count}")
print('------')

brinker['is_negative'] = brinker['stars'] <= 3

# Contamos la cantidad de reseñas negativas y positivas
negative_reviews_count = brinker['is_negative'].sum()
positive_reviews_count = (~brinker['is_negative']).sum()

print(f"Cantidad de reseñas negativas brinker: {negative_reviews_count}")
print(f"Cantidad de reseñas positivas brinker: {positive_reviews_count}")



Cantidad de reseñas negativas bloomin: 1811
Cantidad de reseñas positivas bloomin: 2419
------
Cantidad de reseñas negativas texas: 482
Cantidad de reseñas positivas texas: 783
------
Cantidad de reseñas negativas brinker: 1391
Cantidad de reseñas positivas brinker: 1031


In [13]:
count_darden = len(darden)
count_texas = len(texas)
count_bloomin = len(bloomin)
count_brinker = len(brinker)

total_reviews = count_darden + count_texas + count_bloomin + count_brinker
print(f'total reviews darden, texas, bloomin y brinker : {total_reviews}')

total reviews darden, texas, bloomin y brinker : 15070


*  KPI1 : aumentar puntuacion promedio en yelp 2 puntos trimestralmente
 
formula: ((Puntuación promedio al final del trimestre - Puntuación promedio al inicio del trimestre)

In [14]:
# calcular la diferencia en puntuación promedio entre trimestres
promedio_estrellas_per_quarter['promedio_stars_q_anterior'] = promedio_estrellas_per_quarter['promedio_stars'].shift(1)
promedio_estrellas_per_quarter['aumento_estrellas'] = promedio_estrellas_per_quarter['promedio_stars'] - promedio_estrellas_per_quarter['promedio_stars_q_anterior']

In [15]:
# Calculamos la puntuación promedio al final del trimestre
promedio_final_trimestre = darden.groupby('quarter')['stars'].mean().reset_index()
promedio_final_trimestre.columns = ['quarter', 'promedio_final_trimestre']

# Calculamos la puntuación promedio al inicio del trimestre (trimestre anterior)
promedio_inicio_trimestre = promedio_final_trimestre.copy()
promedio_inicio_trimestre['quarter'] = promedio_inicio_trimestre['quarter'].shift(-1)
promedio_inicio_trimestre.columns = ['quarter', 'promedio_inicio_trimestre']

# Unimos ambas tablas para tener las puntuaciones de inicio y fin de trimestre
quarterly_avg = pd.merge(promedio_final_trimestre, promedio_inicio_trimestre, on='quarter', how='left')


In [16]:
# Unimos ambas tablas para tener las puntuaciones de inicio y fin de trimestre
quarterly_avg = pd.merge(promedio_final_trimestre, promedio_inicio_trimestre, on='quarter', how='left')

In [17]:
quarterly_avg['variacion'] = (quarterly_avg['promedio_final_trimestre'] - quarterly_avg['promedio_inicio_trimestre'])

In [20]:
quarterly_avg.columns = [
    'quarter',
    'promedio al final del trimestre',
    'promedio al inicio del trimestre',
    'variacion puntual'
]

In [21]:
kpi1 = quarterly_avg
kpi1

,quarter,promedio al final del trimestre,promedio al inicio del trimestre,variacion puntual
0,2005Q4,4.000000,NaN,NaN
1,2006Q3,3.800000,4.000000,-0.200000
2,2006Q4,4.000000,3.800000,0.200000
3,2007Q1,5.000000,4.000000,1.000000
4,2007Q2,3.000000,5.000000,-2.000000
...,...,...,...,...
58,2021Q1,3.717391,3.919355,-0.201964
59,2021Q2,3.652174,3.717391,-0.065217
60,2021Q3,3.671329,3.652174,0.019155
61,2021Q4,3.550633,3.671329,-0.120696


* KPI2 : reducir un 7% la cantidad de reviews negativas respecto al semestre anterior

formula : (Cantidad de reseñas negativas del trimestre actual - Cantidad de reseñas negativas del trimestre anterior) / Cantidad de reseñas negativas del trimestre anterior  * 100

In [22]:
# Definimos una reseña negativa como una con una puntuación de 1 a 3 estrellas
darden['negativa'] = darden['stars'] <= 3

# cantidad de reseñas negativas por trimestre
reviews_negativas_per_quarter = darden[darden['negativa']].groupby('quarter').size().reset_index(name='reviews_negativas')

# cantidad de reseñas negativas del trimestre anterior
reviews_negativas_per_quarter['reviews_negativas_quarter_anterior'] = reviews_negativas_per_quarter['reviews_negativas'].shift(1)

# cambio porcentual de las reseñas negativas
reviews_negativas_per_quarter['percentage_change'] = ((reviews_negativas_per_quarter['reviews_negativas_quarter_anterior'] - reviews_negativas_per_quarter['reviews_negativas']) / reviews_negativas_per_quarter['reviews_negativas_quarter_anterior']) * 100

# Renombramos columnas
reviews_negativas_per_quarter.columns = [
    'quarter', 
    'reseñas negativas del trimestre actual', 
    'reseñas negativas del trimestre anterior', 
    'cambio porcentual'
]


In [23]:

kpi2 = reviews_negativas_per_quarter
kpi2

,quarter,reseñas negativas del trimestre actual,reseñas negativas del trimestre anterior,cambio porcentual
0,2006Q3,2,NaN,NaN
1,2007Q2,1,2.0,50.000000
2,2007Q4,1,1.0,0.000000
3,2008Q2,2,1.0,-100.000000
4,2008Q3,2,2.0,0.000000
5,2008Q4,1,2.0,50.000000
6,2009Q1,2,1.0,-100.000000
7,2009Q2,3,2.0,-50.000000
8,2009Q3,6,3.0,-100.000000
9,2009Q4,8,6.0,-33.333333


* KPI3 : aumentar un 5% la puntuacion de 4 y 5 estrellas respecto al trimestre anterior

formula : (n° de reseñas de 4 y 5 estrellas del trimestre actual - n° de reseñas de 4 y 5 estrellas del trimestre anterior) / n° de reseñas de 4 y 5 estrellas del trimestre anterior * 100

In [36]:
darden['positiva'] = darden['stars'] >= 4

# cantidad de reseñas positivas por trimestre
reviews_positivas_per_quarter = darden[darden['positiva']].groupby('quarter').size().reset_index(name='reviews_positivas')

# cantidad de reseñas negativas del trimestre anterior
reviews_positivas_per_quarter['reviews_positivas_quarter_anterior'] = reviews_positivas_per_quarter['reviews_positivas'].shift(1)

# cambio porcentual de las reseñas negativas
reviews_positivas_per_quarter['variacion'] = (
    (reviews_positivas_per_quarter['reviews_positivas'] - reviews_positivas_per_quarter['reviews_positivas_quarter_anterior']) 
    / reviews_positivas_per_quarter['reviews_positivas_quarter_anterior']
) * 100

In [37]:
reviews_positivas_per_quarter

,quarter,reviews_positivas,reviews_positivas_quarter_anterior,variacion
0,2005Q4,1,NaN,NaN
1,2006Q3,3,1.0,200.000000
2,2006Q4,2,3.0,-33.333333
3,2007Q1,1,2.0,-50.000000
4,2007Q2,1,1.0,0.000000
...,...,...,...,...
57,2021Q1,89,84.0,5.952381
58,2021Q2,88,89.0,-1.123596
59,2021Q3,87,88.0,-1.136364
60,2021Q4,95,87.0,9.195402


* KPI4: aumento del 3% en la Cantidad de Reseñas Favorables Frente a la Competencia Respecto al trimestre Anterior

formula: (n° de reseñas favorables del trimestre actual / reseñas favor de la competencia del trimestre actual) - ( n° de reseñas favorables del trimestre anterior /reseñas favor de la competencia del trimestre anterior )

realizo concat para unir los datos de la competencia en un solo dataframe :

In [42]:
competencia = pd.concat([texas, bloomin, brinker], ignore_index=True)

In [43]:
# convertir a formato datetime la columna fecha 
competencia['date'] = pd.to_datetime(competencia ['date'])

In [52]:
# creo una nueva columna que contenga los trimestres del año
competencia['quarter'] = competencia['date'].dt.to_period('Q')

# filtro ambos df para obtener solo las reseñas con 4 y 5 estrellas (reseñas favorables)
darden_favorable = darden[darden['stars'] >= 4]
competencia_favorable = competencia[competencia['stars'] >= 4]

# Filtrar las reseñas de la competencia
#competition_favorable = competencia_favorable[competencia_favorable['marcas'].isin(['texas', 'bloomin', 'brinker'])]

# Agrupar por trimestre y contar el número de reseñas favorables en cada grupo para darden
darden_favorable_per_quarter = darden_favorable.groupby('quarter').size().reset_index(name='darden_favorable_reviews')

# Agrupar por trimestre y contar el número de reseñas favorables en cada grupo para la competencia
competition_favorable_per_quarter = competencia_favorable.groupby('quarter').size().reset_index(name='competencia_favorable_reviews')

# Unir los DataFrames por el semestre
kpi4 = pd.merge(darden_favorable_per_quarter, competition_favorable_per_quarter, on='quarter', how='left')

# Calcular las reseñas favorables del semestre anterior
kpi4['darden_favorable_reviews_previous_quarter'] = kpi4['darden_favorable_reviews'].shift(1)
kpi4['competition_favorable_reviews_previous_quarter'] = kpi4['competencia_favorable_reviews'].shift(1)

# Calcular la proporcion de reseñas favorables frente a la competencia
kpi4['promedio actual'] = kpi4['darden_favorable_reviews'] / kpi4['competencia_favorable_reviews']
kpi4['promedio previo'] = kpi4['darden_favorable_reviews_previous_quarter'] / kpi4['competition_favorable_reviews_previous_quarter']

# Calcular el cambio en la proporción de reseñas favorables
kpi4['variacion promedio'] = kpi4['promedio actual'] - kpi4['promedio previo']
kpi4.columns = [
    'quarter', 
    'reseñas favorables Darden del trimestre actual', 
    'reseñas favorables competencia del timestre actual', 
    'reseñas favorables Darden del trimestre anterior', 
    'reseñas favorables competencia del trimestre anterior', 
    'proporcion actual reseñas favorables', 
    'proporcion anterior reseñas favorables', 
    'variacion'
]

kpi4

,quarter,reseñas favorables Darden del trimestre actual,reseñas favorables competencia del timestre actual,reseñas favorables Darden del trimestre anterior,reseñas favorables competencia del trimestre anterior,proporcion actual reseñas favorables,proporcion anterior reseñas favorables,variacion
0,2005Q4,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2006Q3,3,4.0,1.0,NaN,0.750000,NaN,NaN
2,2006Q4,2,1.0,3.0,4.0,2.000000,0.750000,1.250000
3,2007Q1,1,2.0,2.0,1.0,0.500000,2.000000,-1.500000
4,2007Q2,1,1.0,1.0,2.0,1.000000,0.500000,0.500000
...,...,...,...,...,...,...,...,...
57,2021Q1,89,98.0,84.0,72.0,0.908163,1.166667,-0.258503
58,2021Q2,88,94.0,89.0,98.0,0.936170,0.908163,0.028007
59,2021Q3,87,98.0,88.0,94.0,0.887755,0.936170,-0.048415
60,2021Q4,95,86.0,87.0,98.0,1.104651,0.887755,0.216896


* KPI5: Alcanzar un 95% de Respuestas en las Reseñas Respecto al trimestre Anterior

formula: (n° de reseñas respondidas del trimestre actual / n° de reseñas respondidas del trimestre anterior ) * 100

In [55]:
# Filtrar para obtener solo las reseñas que tienen una respuesta
#respuesta_reviews = darden[darden['text_reply'].notna() & darden['text_reply'].str.strip().ne('')]

# Agrupar por trimestre y contar el número de reseñas respondidas en cada grupo
#respuesta_reviews_per_quarter = respuesta_reviews.groupby('quarter').size().reset_index(name='respuesta_reviews')

# Calcular las reseñas respondidas del trimestre anterior
#respuesta_reviews_per_quarter['respuesta_reviews_anterior_quarter'] = respuesta_reviews_per_quarter['respuesta_reviews'].shift(1)

# Calcular el porcentaje de respuestas respecto al trimestre anterior
#respuesta_reviews_per_quarter['porcentaje_respuesta'] = (
##    respuesta_reviews_per_quarter['respuesta_reviews'] / respuesta_reviews_per_quarter['respuesta_reviews_anterior_quarter']
#) * 100

#respuesta_reviews_per_quarter

,quarter,respuesta_reviews,respuesta_reviews_anterior_quarter,porcentaje_respuesta


Analisis de trimestres: 

In [ ]:
reviews_per_quarter = darden.groupby('quarter').size().reset_index(name='review_count')
reviews_per_quarter

,quarter,review_count
0,2005Q4,1
1,2006Q3,5
2,2006Q4,2
3,2007Q1,1
4,2007Q2,2
...,...,...
58,2021Q1,138
59,2021Q2,138
60,2021Q3,143
61,2021Q4,158
